In [ ]:
from datasets import get_dataset_config_names

xtreme_subsets=get_dataset_config_names("xtreme")
print(f"XTREME 서브셋 개수: {len(xtreme_subsets)}")

In [ ]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

In [ ]:
from datasets import load_dataset

load_dataset("xtreme", name="PAN-X.de")

In [ ]:
from collections import defaultdict
from datasets import DatasetDict

langs=["de", "fr", "it", "en"]
fracs=[0.629, 0.229, 0.084, 0.059]
# 키가 없는 경우 DatasetDict를 반환합니다.
panx_ch=defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # 다국어 말뭉치를 로드한다.
    ds=load_dataset("xtreme", name=f"PAN-X.{lang}")
    # 각 분할을 언어 비율에 따라 다운샘플링하고 섞는다.
    for split in ds:
        panx_ch[lang][split]=(
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac *ds[split].num_rows)))
        )

In [ ]:
import pandas as pd
pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
             index=["Number of training examples"])

In [ ]:
element=panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}:{value}")

In [ ]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

In [ ]:
tags=panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

In [ ]:
def create_tag_names(batch):
    return {
        "ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]
    }

panx_de=panx_ch["de"].map(create_tag_names)

In [ ]:
de_example=panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],["Tokens","Tags"])

In [ ]:
from collections import Counter

split2freqs=defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type=tag.split("-")[1]
                split2freqs[split][tag_type] +=1
pd.DataFrame.from_dict(split2freqs, orient="index")